In [1]:
import pandas as pd
import numpy as np

import torch

from transformers import RobertaTokenizer, RobertaForSequenceClassification

In [2]:
df = pd.read_csv('../data/esnli_train_1.csv') #rmb to train on whole dataset
df.head()

,pairID,gold_label,Sentence1,Sentence2,Explanation_1,WorkerId,Sentence1_marked_1,Sentence2_marked_1,Sentence1_Highlighted_1,Sentence2_Highlighted_1
0,3416050480.jpg#4r1n,neutral,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,the person is not necessarily training his horse,AF0PI3RISB5Q7,A person on a horse jumps over a broken down a...,A person is *training* *his* *horse* for a co...,{},"3,4,5"
1,3416050480.jpg#4r1c,contradiction,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",One cannot be on a jumping horse cannot be a d...,A36ZT2WFIA2HMF,A person *on* *a* *horse* *jumps* over a brok...,"A person *is* *at* *a* *diner,* *ordering* an...","4,2,3,5","2,5,4,3,6"
2,3416050480.jpg#4r1e,entailment,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",a broken down airplane is outdoors,A2GK75ZQTX2RDZ,A person on a horse jumps over *a* *broken* *...,"A person is *outdoors,* on a horse.","8,9,10,7",3
3,2267923837.jpg#2r1n,neutral,Children smiling and waving at camera,They are smiling at their parents,Just because they are smiling and waving at a ...,A18TOIDG32QICP,Children smiling and waving at camera,They are smiling *at* *their* *parents*,{},"5,3,4"
4,2267923837.jpg#2r1e,entailment,Children smiling and waving at camera,There are children present,The children must be present to see them smili...,AEX0YE6TUZRHT,*Children* *smiling* *and* *waving* at camera,There are children *present*,"0,1,3,2",3


In [3]:
def renameColumnsTrain(df):
    return df.rename(columns={'Sentence1': 'premise', 'Sentence2': 'hypothesis', 'Explanation_1': 'explanation'}).drop(["WorkerId", "Sentence1_Highlighted_1", "Sentence2_Highlighted_1"], axis=1)

df_cleaned = renameColumnsTrain(df)

In [4]:
label_to_id = {"entailment": 0, "neutral": 1, "contradiction": 2}
id_to_label = {v: k for k, v in label_to_id.items()}

## Test

In [5]:
# cell for testing the model's output for a single example
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base')

# Tokenize input
premise = df_cleaned['premise'][0]
hypothesis = df_cleaned['hypothesis'][0]
explanation = df_cleaned['explanation'][0]
actual_label = df_cleaned['gold_label'][0]
encoded_input = tokenizer.encode_plus(premise, hypothesis, explanation, padding=True, truncation=True, return_tensors='pt')

labels = torch.tensor(df_cleaned['gold_label'].replace(label_to_id).tolist())[0]
print(encoded_input)
output = model(**encoded_input)

predicted_class = torch.argmax(output.logits, dim=1)

print(f"Premise: {premise}\nHypothesis: {hypothesis}\nExplanation: {explanation}\n")
print(f"True class: {actual_label}")
print(f"Predicted class: {id_to_label[predicted_class.item()]}")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'input_ids': tensor([[    0,   250,   621,    15,    10,  5253, 13855,    81,    10,  3187,
           159, 16847,     4,     2,     2,   250,   621,    16,  1058,    39,
          5253,    13,    10,  1465,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}
Premise: A person on a horse jumps over a broken down airplane.
Hypothesis: A person is training his horse for a competition.
Explanation: the person is not necessarily training his horse

True class: neutral
Predicted class: entailment


Concatenate everything together

# RoBERTa classifier

---

In [6]:
def filterNan(df):
    return df.dropna()

# def tokenize(df):
#     return df.apply(lambda x: tokenizer.encode_plus(x['premise'], x['hypothesis'], x['explanation'], padding='max_length', return_tensors='pt'), axis=1)

def convert_to_tensors(df):
    return torch.tensor(df.values)

def encode_labels(df):
    return df.apply(lambda x: label_to_id[x])

template = """
Premise: {}
Hypothesis: {}
Explanation: {}
"""

def tokenize(df):
    tokenized_batch = []
    for _, row in df.iterrows():
        encoded_dict = tokenizer.encode_plus(
            text = template.format(row['premise'], row['hypothesis'], row['explanation']),
            # row['premise'], # two ways to encode
            # row['hypothesis'], 
            # row['explanation'],
            padding='max_length',
            return_tensors='pt',
            truncation=True
        )
        tokenized_batch.append(encoded_dict)
    return tokenized_batch

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[RoBERTA huggingface](https://huggingface.co/FacebookAI/roberta-base#:~:text=RoBERTa%20is%20a%20transformers%20model%20pretrained%20on%20a,to%20generate%20inputs%20and%20labels%20from%20those%20texts)

In [28]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
# for mac
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# for nvidia GPUs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [30]:
df_cleaned = renameColumnsTrain(df)
df_cleaned = df_cleaned[:100]
df_cleaned = filterNan(df_cleaned)
df_cleaned['gold_label'] = encode_labels(df_cleaned['gold_label'])
tokenized_input = tokenize(df_cleaned)
actual_labels = convert_to_tensors(df_cleaned['gold_label'])

## Predict without training

In [47]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

input_ids = [x['input_ids'].squeeze(0) for x in tokenized_input]
input_ids = pad_sequence(input_ids, batch_first=True)
attention_masks = [x['attention_mask'].squeeze(0) for x in tokenized_input]
attention_masks = pad_sequence(attention_masks, batch_first=True)

labels = actual_labels

dataset = TensorDataset(input_ids, attention_masks, labels)
loader = DataLoader(dataset, batch_size=16)

model.to(device)
model.eval()
predictions = []

with torch.no_grad():
    for batch in loader:
        batch_input_ids, batch_attention_mask, batch_labels = batch

        batch_input_ids = batch_input_ids.to(device)
        batch_attention_mask = batch_attention_mask.to(device)
        
        outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
        logits = outputs.logits.cpu()

        predicted_classes = torch.argmax(logits, dim=1)
        predictions.extend(predicted_classes)

predictions = torch.stack(predictions)

In [48]:
from sklearn.metrics import f1_score

def calc_f1_score(predicted_classes, actual_labels):
    return f1_score(predicted_classes, actual_labels, average='weighted'), f1_score(predicted_classes, actual_labels, average='micro'), f1_score(predicted_classes, actual_labels, average='macro')

print(calc_f1_score(predictions, actual_labels))

(0.36009653092006033, 0.29, 0.2155857214680744)


In [49]:
encoded_input = tokenizer.encode_plus(premise, hypothesis, padding=True, truncation=True, return_tensors='pt')

labels = torch.tensor(df_cleaned['gold_label'].replace(label_to_id).tolist())[0]
output = model(**encoded_input)

predicted_class = torch.argmax(output.logits, dim=1)
print(f"Predicted class: {id_to_label[predicted_class.item()]}")

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [107]:
# test the model
def renameColumnsTest(df):
    return df.rename(columns={'Sentence1': 'premise', 'Sentence2': 'hypothesis', 'Explanation_1': 'explanation'}).drop(["Sentence1_Highlighted_1", "Sentence2_Highlighted_1"], axis=1)

test_data = '../data/esnli_test.csv'
df_test = pd.read_csv(test_data)
df_test_cleaned = renameColumnsTest(df_test)
tokenized_input_test = tokenize(df_test_cleaned)[0]
output = model(**tokenized_input_test)
predicted_class = torch.argmax(output.logits, dim=1)

print(f"Premise: {premise}\nHypothesis: {hypothesis}\nExplanation: {explanation}\n")

print(f"True class: {actual_label}")
print(f"Predicted class: {id_to_label[predicted_class.item()]}")




Premise: A person on a horse jumps over a broken down airplane.
Hypothesis: A person is training his horse for a competition.
Explanation: the person is not necessarily training his horse

True class: 1
Predicted class: entailment


## Finetune model classification head